In [1]:
import datetime
import numpy as np
from matplotlib import cm, pyplot as plt
import pandas as pd
from pandas_datareader import data
import ta

In [2]:
df = data.DataReader("^NSEI",start='2017-01-01',end='2021-05-04', data_source='yahoo')

In [3]:
df.to_csv("nifty.csv")

In [2]:
df=pd.read_csv("nifty.csv")

In [3]:
df.tail()

,Date,High,Low,Open,Close,Volume,Adj Close
1060,2021-04-28,14890.250000,14694.950195,14710.500000,14864.549805,457000.0,14864.549805
1061,2021-04-29,15044.349609,14814.450195,14979.000000,14894.900391,517500.0,14894.900391
1062,2021-04-30,14855.450195,14601.700195,14747.349609,14631.099609,613900.0,14631.099609
1063,2021-05-03,14673.849609,14416.250000,14481.049805,14634.150391,448900.0,14634.150391
1064,2021-05-04,14723.400391,14461.500000,14687.250000,14496.500000,503400.0,14496.500000


In [4]:
df["ho"]=(((df.High-df.Open)/df.Open)*100)

In [5]:
df["ol"]=(((df.Open-df.Low)/df.Open)*100)

In [6]:
df["oc"]=(((df.Open-df.Close)/df.Open)*100)

In [7]:
df["hl"]=(((df.High-df.Low)/df.Low)*100)

In [66]:
# 1 df['rsi']=ta.momentum.rsi(df["Close"], n=14, fillna=False)
# 1 df['r']=ta.momentum.wr(df["High"], df["Low"], df["Close"], lbp=14, fillna=False)
# 1 df['cci']=ta.trend.cci(df["High"], df["Low"], df["Close"], n=20, c=0.015, fillna=False) 

# 2 df['rsi']=ta.momentum.RSIIndicator(df["Close"], window=14, fillna=False)
# 2 df['r']=ta.momentum.WilliamsRIndicator(df["High"], df["Low"], df["Close"], lbp=14, fillna=False)
# 2 df['cci']=ta.trend.CCIIndicator(df["High"], df["Low"], df["Close"], window=20, constant=0.015, fillna=False) 

# 3 df['rsi']=ta.momentum.rsi(df["Close"])
# 3 df['r']=ta.momentum.WilliamsRIndicator(df["High"], df["Low"], df["Close"]).williams_r()
# 3 df['cci']=ta.trend.cci(df["High"], df["Low"], df["Close"])

df['rsi']=ta.momentum.RSIIndicator(df["Close"], window=14, fillna=False).rsi()
df['r']=ta.momentum.WilliamsRIndicator(df["High"], df["Low"], df["Close"], lbp=14, fillna=False).williams_r()
df['cci']=ta.trend.CCIIndicator(df["High"], df["Low"], df["Close"], window=20, constant=0.015, fillna=False).cci()

In [67]:
df["m1"]=(df["Close"]/df["Close"].shift(1))*100
df["m2"]=(df["Close"]/df["Close"].shift(2))*100
df["m3"]=(df["Close"]/df["Close"].shift(3))*100

In [68]:
df["pvc"]=(df["Volume"]-df["Volume"].shift(1))/df["Volume"].shift(1)*100
df["pdc"]=(df["Close"]-df["Close"].shift(1))/df["Close"].shift(1)*100

In [69]:
df['std_3'] = df['pdc'].rolling(5).std()
df['ret_3'] = df['pdc'].rolling(5).mean()

In [70]:
df["nextOpen"]=(df["Open"].shift(-1)-df["Close"])/df["Close"]*100

In [71]:
df["oc1"]=df["oc"].shift(-1)

In [72]:
df["pdc1"]=df["pdc"].shift(-1)

In [73]:
df["Signal"]=np.where(((df['pdc1'] >= 0.5)),"BUY",(np.where(((df['pdc1'] <= -1)),"SELL","HOLD")))

In [74]:
df["Signal"].value_counts()

HOLD    624
BUY     332
SELL    109
Name: Signal, dtype: int64

In [75]:
X=df[['ho','ol','oc','hl','pdc','m1','m2','m3','rsi','r','cci','nextOpen']]
#X=df[['ho','ol','oc','hl','pvc','pdc','m1','m2','m3','rsi','r','cci','nextOpen']]

In [76]:
Y=df[['Signal']]

In [77]:
len(X)

1065

In [78]:
len(Y)

1065

In [79]:
X[1:].isnull().values.any()

True

In [80]:
# Total dataset length
dataset_length = df.shape[0]

# Training dataset length
split = int(dataset_length * 0.8)

split

852

In [81]:
X_train = X[21:split]
X_test = X[split:-2]
Y_train = Y[21:split]
Y_test = Y[split:-2]


In [82]:
# Print the size of the train and test dataset
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

(831, 12) (211, 12)
(831, 1) (211, 1)


In [83]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [84]:
clf = RandomForestClassifier(n_estimators=100,random_state=5)
clf.fit(X_train, Y_train)

c:\users\harsh\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(random_state=5)

In [85]:
p=clf.predict(X_train)

In [86]:
np.unique(p)

array(['BUY', 'HOLD', 'SELL'], dtype=object)

In [88]:
from sklearn.metrics import classification_report
report = classification_report(Y_train, p)
print(report)

              precision    recall  f1-score   support

         BUY       1.00      1.00      1.00       236
        HOLD       1.00      1.00      1.00       518
        SELL       1.00      1.00      1.00        77

    accuracy                           1.00       831
   macro avg       1.00      1.00      1.00       831
weighted avg       1.00      1.00      1.00       831



In [89]:
# Python script for confusion matrix creation. 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
results = confusion_matrix(Y_train, p) 
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(Y_train, p))


Confusion Matrix :
[[236   0   0]
 [  0 518   0]
 [  0   0  77]]
Accuracy Score : 1.0


In [90]:
t=clf.predict(X_test)

In [91]:
np.unique(t)

array(['BUY', 'HOLD', 'SELL'], dtype=object)

In [92]:
results = confusion_matrix(Y_test, t) 
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(Y_test, t))


Confusion Matrix :
[[30 59  0]
 [18 71  2]
 [ 3 26  2]]
Accuracy Score : 0.4881516587677725


In [93]:
report = classification_report(Y_test, t)
print(report)

              precision    recall  f1-score   support

         BUY       0.59      0.34      0.43        89
        HOLD       0.46      0.78      0.57        91
        SELL       0.50      0.06      0.11        31

    accuracy                           0.49       211
   macro avg       0.51      0.39      0.37       211
weighted avg       0.52      0.49      0.45       211



In [94]:
clf.feature_importances_

array([0.07983384, 0.06829465, 0.07078411, 0.08316294, 0.0639127 ,
       0.06713321, 0.07297042, 0.07087711, 0.08857615, 0.07448443,
       0.08206322, 0.17790721])

In [95]:
t

array(['HOLD', 'HOLD', 'HOLD', 'HOLD', 'BUY', 'HOLD', 'HOLD', 'HOLD',
       'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'BUY', 'BUY', 'HOLD',
       'BUY', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD',
       'BUY', 'HOLD', 'HOLD', 'BUY', 'BUY', 'BUY', 'HOLD', 'BUY', 'HOLD',
       'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'BUY', 'HOLD', 'HOLD',
       'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'BUY', 'HOLD',
       'HOLD', 'HOLD', 'BUY', 'HOLD', 'HOLD', 'BUY', 'HOLD', 'BUY',
       'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'BUY', 'BUY', 'SELL',
       'HOLD', 'BUY', 'HOLD', 'BUY', 'HOLD', 'HOLD', 'HOLD', 'HOLD',
       'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'BUY', 'HOLD',
       'HOLD', 'BUY', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD',
       'HOLD', 'BUY', 'HOLD', 'HOLD', 'BUY', 'HOLD', 'BUY', 'HOLD',
       'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'BUY', 'HOLD',
       'HOLD', 'HOLD', 'BUY', 'HOLD', 'BUY', 'HOLD', 'BUY', 'HOLD',
       'HOLD', 'HOLD', 'H